In [ ]:
import pandas as pd



In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df2 = df.dropna()

In [ ]:
df2.shape

In [ ]:
df2['prix'] = df2['prix'].apply(lambda x: int(str(x).replace(',', '')))

In [ ]:
df2[df2['prix'] < 100000].count()

In [ ]:
df2 = df2[df2['prix'] >= 100000]

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
df2['city'] = df2['ville'].str.extract(r'dans\s+([^,]+)')

In [ ]:
df2.head(50)

In [ ]:
df2['secteur'] = df2['ville'].str.extract(r',\s*([^,]+)(?:\s*,|$)')

In [ ]:
df2.head(50)

In [ ]:
df2 = df2.drop('ville', axis=1)
df2 = df2.rename(columns={'city': 'ville'})

In [ ]:
df2.head()

In [ ]:
df2['superficie'] = df2['superficie'].str.extract('(\d+)').fillna(0).astype(int)

In [ ]:
df2.tail()

In [ ]:
df2.shape

In [ ]:
# Handle '10+ ' case and convert to int
df2['chambres'] = df2['chambres'].apply(lambda x: int(x.replace('+', '').strip()))
df2['salles_de_bain'] = df2['salles_de_bain'].apply(lambda x: int(x.replace('+', '').strip()))

In [ ]:
df2 = df2.rename(columns={'superficie': 'superficie en m²'})

In [ ]:
df2.head()

In [ ]:
df2[df2['superficie en m²'] == 0].count()

In [ ]:
df2 = df2[df2['superficie en m²'] != 0]

In [ ]:
df2.shape

In [ ]:
df2[df2['prix'] == 2000000000]

In [ ]:
df3 = df2.groupby('secteur')['secteur'].agg('count').sort_values(ascending=False)

In [ ]:
df2 = df2.drop('secteur', axis=1)

In [ ]:
df2.head()

In [ ]:
df4 = df2.groupby('type')['type'].agg('count')
df4.head()

In [ ]:
villa_count = df2[
   ((df2['titre'].str.contains('villa', case=False)) | 
    (df2['titre'].str.contains('فيلا', case=False))) & 
   (df2['type'] == 'Maison et Villa')
].shape[0]

villa_count

In [ ]:
df2.loc[
   ((df2['titre'].str.contains('villa', case=False)) | 
    (df2['titre'].str.contains('فيلا', case=False))) & 
   (df2['type'] == 'Maison et Villa'),
   'type'
] = 'Villa et Riad'

In [ ]:
df3 = df2.groupby('type')['type'].agg('count')

In [ ]:
df3.head(50)

In [ ]:
df2 = df2[df2['superficie en m²'] >= 10]

In [ ]:
df2[df2['superficie en m²'] ==13]

In [ ]:
df2.shape

In [ ]:
df2[df2['superficie en m²'] == 25]

In [ ]:
mask = df2['titre'].str.contains(r'\d+\s*m²', case=False, na=False)
df2.loc[mask, 'superficie en m²'] = df2.loc[mask, 'titre'].str.extract(r'(\d+)\s*m²')[0].fillna(df2.loc[mask, 'superficie en m²']).astype(int)

In [ ]:
 df2[(df2['superficie en m²'] == 46) & (df2['prix'] ==350000 	)]

In [ ]:
df2 = df2.drop('titre', axis=1)

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
df2[(df2['prix'] > 5000000) & (df2['superficie en m²'] < 200) & (df2['type'] == 'appartement')]

In [ ]:
mask = (df2['prix'] > 5000000) & (df2['superficie en m²'] < 200) & (df2['type'] == 'appartement')
df2 = df2[~mask]

In [ ]:
df2.shape

In [ ]:
mask = (df2['prix'] > 5000000) & (df2['type'] == 'appartement')
df2 = df2[~mask]

In [ ]:
df2.shape

In [ ]:
mask = (df2['prix'] < 200000) & (df2['type'] == 'appartement')
df2 = df2[~mask]
df2.shape

In [ ]:
mask = (df2['prix'] < 300000) & (df2['superficie en m²'] >60) & (df2['type'] == 'appartement')
df2 = df2[~mask]

In [ ]:
df2.shape

In [ ]:
maskk = (df2['prix'] > 5000000) & (df2['type'] == 'Maison et Villa')
df2 = df2[~maskk]

In [ ]:
df2.shape

In [ ]:
maskk = (df2['prix'] < 500000) & (df2['superficie en m²'] > 40) & (df2['type'] == 'Maison et Villa')
df2 = df2[~maskk]

In [ ]:
df2.shape

In [ ]:
maskk = (df2['prix'] < 300000) & (df2['type'] == 'Maison et Villa')
df2 = df2[~maskk]

In [ ]:
df2.shape

In [ ]:
maskt = (df2['prix'] < 1000000) & (df2['type'] == 'Villa et Riad') 
df2 = df2[~maskt]

In [ ]:
maskt2 = (df2['prix'] > 20000000) & (df2['type'] == 'Villa et Riad') 
df2 = df2[~maskt2]

In [ ]:
df2.shape

In [ ]:
df7 = df2.groupby('type')['type'].agg('count')

In [ ]:
ville_counts = df2['ville'].value_counts()
valid_villes = ville_counts[ville_counts >= 20].index
valid_villes

In [ ]:
df2['ville'].value_counts()

In [ ]:
df2 = df2[df2['ville'].isin(valid_villes)]
df2.shape
df2['ville'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import pickle
# More aggressive outlier removal
def remove_outliers(df, price_column='prix'):
    Q1 = df[price_column].quantile(0.05)
    Q3 = df[price_column].quantile(0.95)
    df = df[(df[price_column] >= Q1) & (df[price_column] <= Q3)]
    
    # Remove outliers from superficie
    Q1_sup = df['superficie en m²'].quantile(0.05)
    Q3_sup = df['superficie en m²'].quantile(0.95)
    return df[(df['superficie en m²'] >= Q1_sup) & (df['superficie en m²'] <= Q3_sup)]

# Clean data
df_cleaned = remove_outliers(df2)
print(f"Original shape: {df2.shape}, After removing outliers: {df_cleaned.shape}")

# Safe feature engineering
def safe_division(a, b):
    return np.where(b > 0, a / b, 0)

# Feature engineering
df_cleaned['rooms_per_bath'] = safe_division(df_cleaned['chambres'], df_cleaned['salles_de_bain'])

# Remove any remaining infinite values
df_cleaned = df_cleaned.replace([np.inf, -np.inf], np.nan).dropna()

# Define features
numeric_features = ['chambres', 'salles_de_bain', 'superficie en m²', 
                    'rooms_per_bath']
categorical_features = ['type', 'ville']

# Create preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ])

# Create pipeline with tuned XGBoost
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.03,
        max_depth=6,
        min_child_weight=2,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        tree_method='hist'
    ))
])

# Prepare data
X = df_cleaned[numeric_features + categorical_features]
y = df_cleaned['prix']

# Log transform target variable
y = np.log1p(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model.fit(X_train, y_train)

# Make predictions and inverse transform
y_pred = np.expm1(model.predict(X_test))
y_test = np.expm1(y_test)

# Calculate metrics
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Model R² Score: {r2}")
print(f"Mean Absolute Percentage Error: {mape*100:.2f}%")

# Print some predictions
print("\nSample Predictions vs Actual:")
for pred, actual in zip(y_pred[:5], y_test[:5]):
    print(f"Predicted: {pred:,.0f}, Actual: {actual:,.0f}")
    print(f"Difference: {abs(pred-actual):,.0f} ({abs(pred-actual)/actual*100:.1f}%)")

# Save model
with open('model_abdo.pkl', 'wb') as f:
    pickle.dump((model, numeric_features, categorical_features), f)

In [ ]:
# Print data statistics
print("\nTraining Data Statistics:")
print("\nNumerical Features Statistics:")
print(df_cleaned[numeric_features].describe())

print("\nPrice Distribution by Type:")
for t in df_cleaned['type'].unique():
    prices = df_cleaned[df_cleaned['type'] == t]['prix']
    print(f"\n{t}:")
    print(f"Min: {prices.min():,.0f}")
    print(f"Max: {prices.max():,.0f}")
    print(f"Mean: {prices.mean():,.0f}")
    print(f"Median: {prices.median():,.0f}")

print("\nPrice Distribution by City (Top 5 cities):")
for city in df_cleaned['ville'].value_counts().head().index:
    prices = df_cleaned[df_cleaned['ville'] == city]['prix']
    print(f"\n{city}:")
    print(f"Min: {prices.min():,.0f}")
    print(f"Max: {prices.max():,.0f}")
    print(f"Mean: {prices.mean():,.0f}")
    print(f"Median: {prices.median():,.0f}")

# Print a few sample rows
print("\nSample Rows:")
print(df_cleaned.head())